In [ ]:
!pip install 'farm-haystack[inference]' 'qdrant-haystack<2.0.0' > /dev/null 2>&1

In [ ]:
# SET UP CONTANT VARIABLES
#####################################################################################
QDRANTDB_URL = "QDRANTDB_URL"
API_KEY = "API_KEY"
DB_BATCH_SIZE = 256
DB_TIMEOUT = 60

EMBEDDING_MODEL = "bkai-foundation-models/vietnamese-bi-encoder"
EMBEDDING_DIM = 768
EMBEDDING_MAX_LENGTH = 256
EMBEDDING_TOP_K = 10

FAQ_FILE = "FAQ_FILE"
FAQ_THRESHOLD = 80

In [ ]:
# DELETE DATABASE
#####################################################################################
# !curl -X DELETE "{QDRANTDB_URL}/collections/faq" --header "api-key: {API_KEY}"

{"result":true,"status":"ok","time":0.052404978}

In [ ]:
# INITIALIZE VECTOR DATABASE
#####################################################################################
import pandas as pd
from tqdm import tqdm
from haystack.schema import Document
from haystack.nodes import PreProcessor
from qdrant_haystack import QdrantDocumentStore

document_store = QdrantDocumentStore(
            url=QDRANTDB_URL,
            api_key=API_KEY,
            embedding_dim=EMBEDDING_DIM,
            timeout=DB_TIMEOUT,
            embedding_field="embedding",
            hnsw_config={"m": 128, "ef_construct": 100},
            similarity="cosine",
            recreate_index=True,
        )

processor = PreProcessor(
        clean_empty_lines=True,
        clean_whitespace=True,
        clean_header_footer=True,
        remove_substrings=None,
        split_by="passage",
        split_length=1,
        split_respect_sentence_boundary=False,
        split_overlap=0,
        max_chars_check=10000,
    )

faq_df = pd.read_csv(FAQ_FILE)
faq_documents = []
idx = 0

for _, d in tqdm(faq_df.iterrows(), desc="Loading FAQ...", total=faq_df.shape[0]):
    content = d["query"]
    faq_documents.append(
        Document(content=content, id=idx, meta={"answer": d["answer"]})
    )
    idx += 1

faq_documents = processor.process(faq_documents)

document_store.write_documents(
    documents=faq_documents,
    index="faq",
    batch_size=DB_BATCH_SIZE,
)

Preprocessing: 100%|██████████| 21515/21515 [00:02<00:00, 9397.99docs/s]
21760it [02:36, 139.32it/s]


In [ ]:
# UPDATE FAQ EMBEDDING TO DATABASE
#####################################################################################
from haystack.nodes import EmbeddingRetriever

embedding_retriever = EmbeddingRetriever(
    document_store=document_store,
    embedding_model=EMBEDDING_MODEL,
    model_format="sentence_transformers",
    top_k=EMBEDDING_TOP_K,
    max_seq_len=EMBEDDING_MAX_LENGTH
)

print("Update FAQ Embedding...")
document_store.update_embeddings(
    embedding_retriever,
    index="faq",
    batch_size=DB_BATCH_SIZE,
    update_existing_embeddings=True
)

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/123 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/6.46k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


...

Updating embeddings:  99%|█████████▉| 21248/21515 [08:25<00:05, 46.83 Docs/s]

Batches:   0%|          | 0/8 [00:00<?, ?it/s]

Updating embeddings: 100%|█████████▉| 21504/21515 [08:31<00:00, 45.68 Docs/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Updating embeddings: 21760 Docs [08:32, 42.45 Docs/s]


In [ ]:
# CHECK DATABASE
#####################################################################################
documents = document_store.get_all_documents(index="faq", batch_size=DB_BATCH_SIZE)

first_document = documents[0] if documents else None

if first_document:
    print("id:", first_document.id)
    print("content:", first_document.content)
    print("embedding vector:", first_document.embedding)
else:
    print("Fail!")

id: 2c36c733d31bf7fc8f7796f27416d65f
content: Tôi có thể liên hệ với Văn phòng Đào tạo Quốc tế qua cách nào tại trường ĐHBK
embedding vector: [ 8.79344500e-03  3.44853440e-02 -6.21258400e-02  5.13360170e-02
 -1.12541370e-02 -3.84084770e-02 -2.90907510e-02  4.66916370e-02
  3.18517000e-02  1.83993450e-02  1.34879830e-02  2.35489570e-02
 -6.11752830e-03 -3.65561900e-02 -8.99884000e-03 -4.36302800e-02
  4.51449800e-02  2.18123700e-02 -1.83146060e-02  2.75568530e-02
  1.18577590e-02 -1.27040260e-02  2.02801080e-02  1.65020800e-02
 -2.64505520e-02 -3.81196100e-03  3.41722480e-02  8.92468300e-02
  2.64163180e-02 -7.49187360e-03 -6.36263660e-03  7.88195300e-03
 -2.38365340e-02  1.25753370e-02 -7.08824400e-02  6.25676500e-02
 -2.07765850e-02  1.64421390e-02  8.76547800e-03  3.91151870e-02
  2.74467100e-02  3.34097450e-02 -5.46978800e-02 -1.96333360e-02
 -6.05066260e-03 -1.25524770e-02 -6.43536160e-04 -1.40424080e-02
 -2.45601070e-02 -9.72049700e-03  5.00777000e-02 -1.36910300e-02
 -1.67675990e

In [ ]:
# SET UP FAQ THRESHOLD
#####################################################################################
from haystack.schema import Document
from typing import Optional, List, Dict, Tuple
from haystack.nodes.base import BaseComponent

class DocumentThreshold(BaseComponent):
    def __init__(
        self,
        threshold: Optional[int] = 75,
    ):
        """
        This component filters documents based on a threshold percentage, ensuring only the
        documents above the threshold get passed down the pipeline.
        :param threshold: The threshold to use for filtering documents in whole numbers (81 = 81%).
        """

        super().__init__()
        self.threshold = float(threshold / 100)

    outgoing_edges = 1

    def run(self, documents: List[Document]) -> Tuple[Dict, str]:
        return {
            "documents": [x for x in documents if x.score >= self.threshold]
        }, "output_1"

    def run_batch(self, results, **kwargs):
        pass

In [ ]:
# TEST RETRIEVER FOR HCMUT CHATBOT
#####################################################################################
from haystack import Pipeline
from haystack.nodes import EmbeddingRetriever, Docs2Answers
from qdrant_haystack import QdrantDocumentStore
from haystack.schema import Document
from haystack.nodes import PreProcessor
from typing import List

processor = PreProcessor(
    clean_empty_lines=True,
    clean_whitespace=True,
    clean_header_footer=True,
    remove_substrings=None,
    split_by="passage",
    split_length=1,
    split_respect_sentence_boundary=False,
    split_overlap=0,
    max_chars_check=10000,
)

document_store = QdrantDocumentStore(
    url=QDRANTDB_URL,
    api_key=API_KEY,
    embedding_dim=EMBEDDING_DIM,
    timeout=DB_TIMEOUT,
    embedding_field="embedding",
    hnsw_config={"m": 128, "ef_construct": 100},
    similarity="cosine",
    recreate_index=False,
)

embedding_retriever = EmbeddingRetriever(
    document_store=document_store,
    embedding_model=EMBEDDING_MODEL,
    model_format="sentence_transformers",
    top_k=EMBEDDING_TOP_K,
    max_seq_len=EMBEDDING_MAX_LENGTH
)

faq_threshold = DocumentThreshold(threshold=FAQ_THRESHOLD)
docs2answers = Docs2Answers()

pipeline = Pipeline()
pipeline.add_node(component=embedding_retriever, name="EmbeddingRetriever", inputs=["Query"])
pipeline.add_node(component=faq_threshold, name="Threshold", inputs=["EmbeddingRetriever"])
pipeline.add_node(component=docs2answers, name="Answer", inputs=["Threshold"])

def preprocess_query(query: str) -> str:
    docs = [Document(content=query)]
    processed_docs = processor.process(docs)
    processed_query = ' '.join([doc.content for doc in processed_docs])
    return processed_query

def query_pipeline(query: str) -> List[Document]:
    preprocessed_query = preprocess_query(query)
    results = pipeline.run(query=preprocessed_query, params={"EmbeddingRetriever": {'index': 'faq'}})
    return results['documents']

In [ ]:
query = "Học phí ngành Khoa học Máy tính bao nhiêu"
documents = query_pipeline(query)

if not documents:
    print("No documents found.")
else:
    for idx, doc in enumerate(documents, start=1):
        print(f"Top {idx}:")
        print(f"Content: {doc.content}")
        print(f"Score: {doc.score}")
        print(f"Meta: {doc.meta}")
        print("-" * 50)

Preprocessing: 100%|██████████| 1/1 [00:00<00:00, 2651.27docs/s]


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Top 1:
Content: Ngành Khoa học máy tính và Kỹ thuật máy tính có học phí là bao nhiêu?
Score: 0.92384135
Meta: {'_split_id': 0, 'answer': 'Tham khảo quy định về mức thu học phí năm học 2023-2024:\r\nhttps://hcmut.edu.vn/dao-tao/hoc-phi\n\r\n1.Chính quy chương trình tiêu chuẩn (gồm cả bằng 2, liên thông cao đẳng -\r\nđại học)\r\n- Dự kiến học phí toàn khóa (ĐVT: VNĐ/năm):\n\r\n|           | **2023 - 2024** | **2024 - 2025** | **2025 - 2026** | **2026 - 2027** |\r\n|:---------:|:---------------:|:---------------:|:---------------:|:---------------:|\r\n| Khóa 2020 |    11.700.000   |                 |                 |                 |\r\n| Khóa 2021 |    24.000.000   |    29.000.000   |                 |                 |\r\n| Khóa 2022 |    27.500.000   |    29.000.000   |    30.000.000   |                 |\r\n| Khóa 2023 |    27.500.000   |    29.000.000   |    30.000.000   |    31.500.000   |\n\r\n2. Chương trình CTTA, CTTT, CTQT, LKQT, CTNB, CTHNB\r\n- Học phí các lớp Pre: 28.600.0